# End-to-End Machine Learning for Aeromagnetic Compensation Notebook
This notebook provides a background on machine learning with an application toward aeromagnetic compensation using MagNav.jl: https://github.com/MIT-AI-Accelerator/MagNav.jl

Feel free to change any parameters of interest.

Machine learning projects entail much more than just training neural networks. In [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/), Aurélien Géron lays out the steps of an end-to-end machine learning project:

1. Look at the big picture
2. Get the data
3. Discover and visualize the data to gain insights
4. Prepare the data for machine learning algorithms
5. Select a model and train it
6. Fine-tune the model
7. Present the solution
8. Launch, monitor, and maintain the system

## 0. Import packages and DataFrames

The DataFrames listed below provide useful information about the flight data (collected by Sander Geophysics Ltd. (SGL) in 2020) and magnetic anomaly maps.

Dataframe  | Description
:--------- | :----------
`df_map`   | map files relevant for SGL flights
`df_cal`   | SGL calibration flight lines
`df_flight`| SGL flight files
`df_all`   | all flight lines
`df_nav`   | all *navigation-capable* flight lines
`df_event` | pilot-recorded in-flight events

In [ ]:
cd(@__DIR__)
# uncomment line below to use local MagNav.jl (downloaded folder)
# using Pkg; Pkg.activate("../"); Pkg.instantiate()
using MagNav
using CSV, DataFrames
using Plots: plot, plot!
using Random: seed!
using Statistics: mean, median, std
seed!(33); # for reproducibility
include("dataframes_setup.jl"); # setup DataFrames


## 1. Look at the big picture

Airborne magnetic anomaly navigation (MagNav) is an emerging technology that can be used for aerial navigation in case GPS is not available. MagNav uses maps of variations in the magnetic field originating from the crust of the Earth. A navigation algorithm compares onboard measurements of the magnetic field with a magnetic anomaly map, which (combined with inertial measurements), produces an estimate of the aircraft position.

But there's a catch! The magnetometers on the aircraft measure the total magnetic field, which is comprised of multiple magnetic fields arising from not only the crust, but also the Earth's core, diurnal variations, and the aircraft itself. In order to use the crustal anomaly field for MagNav, the other contributions to the total magnetic field must be removed. Magnetic models and base station measurements suffice to remove the core and diurnal fields, which leaves the aircraft field to remove. Unlike the other contributions, the aircraft field is difficult to isolate. Aeromagnetic compensation is used to identify and remove the aircraft field.

The standard approach for aeromagnetic compensation, known as the Tolles-Lawson model, uses a physics-based linear model of the aircraft field combined with data taken during a specific flight pattern designed to maximize the contribution arising from the aircraft. Tolles-Lawson works well when the aircraft field is small compared to the Earth's core field, for example when the magnetometer is located on a boom (stinger) behind the aircraft (Mag 1) but falls short for magnetometers located in the cabin (Mags 2-5).

The goal here is to perform aeromagnetic compensation using the in-cabin sensors. In addition to the scalar magnetometers (Mags 2-5), which detect the magnitude of the total magnetic field, there are measurements from vector magnetometers (Flux A-D), which detect the three cartesian components of the total magnetic field. There are also measurements available from additional sensors, notably current sensors. Performance is measured using the standard deviation of the error between the predicted values and the professionally-compensated stinger magnetometer.

<img src="../readmes/magnetometer_locations.png" alt="magnetometer locations" />

## 2. Get the data

For Tolles-Lawson and testing, we select Flight 1006 (see [readme](https://github.com/MIT-AI-Accelerator/MagNav.jl/blob/master/readmes/Flt1006_readme.txt)) and gather the [`XYZ20` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#MagNav.XYZ20), which contains the GPS-based trajectory [`Traj` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#MagNav.Traj), inertial navigation system [`INS` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#MagNav.INS), flight information, magnetometer readings, and auxilliary sensor data.

In [ ]:
flight = :Flt1006 # select flight, full list in df_flight
xyz    = get_XYZ(flight,df_flight); # load flight data


The `xyz` flight data struct is of type `MagNav.XYZ20` (for the 2020 SGL flight data collection), which is a subtype of `MagNav.XYZ` (the abstract type for any flight data in MagNav.jl). There are 76 fields, which can be accessed using dot notation. Note that `xyz` holds all the flight data from the HDF5 file, but Boolean indices can be used as a mask to return specific portion(s) of flight data.

In [ ]:
typeof(xyz)


In [ ]:
fieldnames(MagNav.XYZ20)


For the Tolles-Lawson calibration, we will select flight line 1006.04, which occurred at a higher altitude (see [readme](https://github.com/MIT-AI-Accelerator/MagNav.jl/blob/master/readmes/Flt1006_readme.txt)). This is the first calibration box of this flight line. `TL_ind` holds the Boolean indices (mask) just for this portion of the calibration flight line. The full list of calibration flight line options is in `df_cal`.

In [ ]:
TL_i   = 6 # select first calibration box of 1006.04
TL_ind = get_ind(xyz;tt_lim=[df_cal.t_start[TL_i],df_cal.t_end[TL_i]]);


Here `df_all` is filtered into `df_options` to ensure that the selected flight line(s) for testing correspond with the selected flight (`:Flt1006`). The full list of SGL flights is in `df_flight` and the full list of flight lines is in `df_all`.

In [ ]:
df_options = df_all[(df_all.flight .== flight),:]


For testing, we will use Boolean indices (mask) corresponding to flight line 1006.08 in `df_options`.

In [ ]:
line = 1006.08 # select flight line (row) from df_options
ind  = get_ind(xyz,line,df_options); # get Boolean indices


For training, we select all available flight data from Flights 1003-1006 (see [readmes](https://github.com/MIT-AI-Accelerator/MagNav.jl/tree/master/readmes)) into `lines_train`, except the held-out flight `line`. To reduce memory use, the specified flight data is only loaded internally during neural network training later on.

In [ ]:
flts = [:Flt1003,:Flt1004,:Flt1005,:Flt1006] # select flights for training
df_train = df_all[(df_all.flight .∈ (flts,) ) .& # use all flight data
                  (df_all.line   .!= line),:]    # except held-out line
lines_train = df_train.line # training lines


## 3. Discover and visualize the data to gain insights

As noted in the [datasheet](https://github.com/MIT-AI-Accelerator/MagNav.jl/blob/master/readmes/datasheet_sgl_2020_train.pdf), the full SGL 2020 training dataset has 753573 total instances (time steps) sampled at 10 Hz, about 21 hours in flight time spread across 6 flights. This notebook looks at Flight 1006 in more detail for testing, which has 108318 instances, or about 3 hours of flight. The held-out flight `line` subset of Flight 1006 has 8391 instances, or about 14 minutes of flight.

To get an idea of the magnetometer data, we can call some utility functions for plotting.

Note that these are filtered using the `ind` Boolean indices corresponding to the held-out flight `line`.

In [ ]:
show_plot = true
save_plot = false
use_mags  = [:mag_1_uc,:mag_4_uc,:mag_5_uc] # scalar magnetometers to plot

p1 = plot_mag(xyz;ind,show_plot,save_plot, # plot scalar magnetometers
              use_mags     = use_mags,
              detrend_data = false);


In [ ]:
println("Mag 1")
describe(xyz.mag_1_uc[ind])


In [ ]:
println("Mag 4")
describe(xyz.mag_4_uc[ind])


In [ ]:
println("Mag 5")
describe(xyz.mag_5_uc[ind])


These scalar magnetometers range between approximately 50510 nT and 53648 nT, and Mag 4 clearly has a bias compared to the others. Note that MagNav relies on magnetic field fluctuations, so a bias (DC offset) in the magnetometer data has limited impact on navigation performance. Compare the plot of the scalar magnetometer values (above) with the mean-subtracted values (below). The same variations can be seen, but the signals are easier to compare. Here it is apparent that the location of the magnetometer is an important factor for the noise seen in the signal.

In [ ]:
p2 = plot_mag(xyz;ind,show_plot,save_plot, # plot scalar magnetometers
              use_mags     = use_mags,
              detrend_data = true);


In [ ]:
p3 = plot_mag(xyz;ind,show_plot,save_plot, # plot vector magnetometer (fluxgate)
              use_mags     = [:flux_d], # try changing to :flux_a, :flux_b, :flux_c
              detrend_data = true);


In [ ]:
println("Flux D total field")
describe(xyz.flux_d.t[ind])


Each vector magnetometer (e.g., `:flux_d`) has four channels: the `_x`, `_y`, and `_z` components and the magnitude or total field `t`. The total field for `Flux D` is between approximately 51579 nT and 54445 nT, a similar range as the scalar magnetometers. However, the components vary between approximately -53190 nT and +38542 nT (when not detrended). This suggests directional information is a key component of the vector magnetometers, so feature scaling should take this into account.

**Overall, it is clear that the in-cabin scalar and vector magnetometers are noisy compared to the stinger magnetometer (Mag 1).**

The dataset also includes additional sensors. For example, a current sensor for the 💡 strobe lights 💡 has been found to be a helpful feature. This current sensor contains high-frequency noise, so first a low-pass filter is applied using two convenience functions, `get_bpf` and `bpf_data`.

In [ ]:
lpf     = get_bpf(;pass1=0.0,pass2=0.2,fs=10.0) # get low-pass filter
lpf_sig = -bpf_data(xyz.cur_strb[ind];bpf=lpf); # apply low-pass filter, sign switched for easier comparison


For comparison, the (linear) Tolles-Lawson model will be evaluated here. We select scalar and vector magnetometer readings during the calibration flight and generate the coefficients to perform linear Tolles-Lawson compensation. We are choosing in-cabin scalar magnetometer 4 and vector (flux) magnetometer D. Mag 4 is located on the floor in the rear of the cabin, and Flux D is nearby on the starboard side. Mag 4 is particularly challenging since it contains several 100s to 1000 nT excursions in comparison to the tail stinger, as we saw above.

### Tolles-Lawson calibration

In [ ]:
λ       = 0.025   # ridge parameter for ridge regression
use_vec = :flux_d # selected vector (flux) magnetometer
terms_A = [:permanent,:induced,:eddy] # Tolles-Lawson terms to use
flux    = getfield(xyz,use_vec) # load Flux D data
TL_d_4  = create_TL_coef(flux,xyz.mag_4_uc,TL_ind; # create Tolles-Lawson
                         terms=terms_A,λ=λ);       # coefficients with Flux D & Mag 4


### Tolles-Lawson compensation

In [ ]:
A = create_TL_A(flux,ind)     # Tolles-Lawson `A` matrix for Flux D
mag_1_sgl = xyz.mag_1_c[ind]  # professionally compensated tail stinger, Mag 1
mag_4_uc  = xyz.mag_4_uc[ind] # uncompensated Mag 4
mag_4_c   = mag_4_uc - detrend(A*TL_d_4;mean_only=true); # compensated Mag 4


Now the low-pass filtered strobe light current sensor and Mag 4 with Tolles-Lawson compensation can be compared. Here we can see that the spikes seem to line up!

In [ ]:
p4 = plot_basic(xyz.traj.tt[ind],lpf_sig;lab="strobe current");
p5 = plot_basic(xyz.traj.tt[ind],mag_4_c;lab="Mag 4");


We can also look at a correlation scatter plot between the low-pass filtered strobe light current sensor and Mag 4 with Tolles-Lawson compensation. It looks like there is a positive correlation between these features. This looks like a fascinating topic for further study.

In [ ]:
p6 = plot_correlation(mag_4_c,lpf_sig,:mag_4_c,:lpf_cur_strb);


Note that the same correlation is **not** seen between the low-pass filtered strobe light current sensor and the uncompensated magnetometers.

In [ ]:
p7 = plot_correlation_matrix(xyz,ind,[:mag_1_uc,:mag_4_uc,:mag_5_uc,:lpf_cur_strb]);


## 4. Prepare the data for machine learning algorithms

Now we attempt to improve on the Tolles-Lawson (TL) model by training an artificial neural network (NN). The NN is provided with in-cabin magnetometer and current sensor measurements as features.

In [ ]:
features = [:mag_4_uc, :TL_A_flux_d, :lpf_cur_com_1, :lpf_cur_strb, :lpf_cur_outpwr, :lpf_cur_ac_lo];


Additional parameters must be set to prepare the training (and testing) data:
- `y_type`: `y` target type, which has multiple options:
    - `:a` = anomaly field #1, compensated tail stinger total field scalar magnetometer measurements
    - `:b` = anomaly field #2, interpolated magnetic anomaly map values
    - `:c` = aircraft field #1, difference between uncompensated cabin total field scalar magnetometer measurements and interpolated magnetic anomaly map values
    - `:d` = aircraft field #2, difference between uncompensated cabin and compensated tail stinger total field scalar magnetometer measurements
    - `:e` = BPF'd total field, bandpass filtered uncompensated cabin total field scalar magnetometer measurements
- `use_mag`:     uncompensated scalar magnetometer to use for `y` target vector
- `sub_diurnal`: if true, subtract diurnal from scalar magnetometer measurements
- `sub_igrf`:    if true, subtract IGRF from scalar magnetometer measurements
- `norm_type_x`: normalization for `x` data matrix
- `norm_type_y`: normalization for `y` target vector

In [ ]:
y_type      = :d
use_mag     = :mag_4_uc
sub_diurnal = true
sub_igrf    = true
norm_type_x = :standardize
norm_type_y = :standardize;


The `y_type` and `use_mag` parameters are set to use the difference between an uncompensated cabin magnetometer (`mag_4_uc`) and the compensated tail stinger magnetometer (`mag_1_c`). This is essentially the aircraft field corruption that needs to be removed for a clean signal.

The `sub_diurnal` and `sub_igrf` parameters are set to remove the diurnal and core fields from all (scalar) total field measurements. This leaves only the anomaly field (desired) and aircraft field (corruption) in those measurements.

The `norm_type_x` and `norm_type_y` parameters are set to standardize the data, both the `x` data matrix and `y` target vector. Most machine learning algorithms perform better when the features are scaled to values close to one. Two of the most common methods are standardization (Z-score normalization) and min-max normalization.

## 5. Select a model and train it

MagNav.jl has a built-in pipeline for training and testing various aeromagnetic compensation models. However, the model parameters must be selected:

- `model_type`: aeromagnetic compensation model type
- `η_adam`: learning rate for Adam optimizer
- `epoch_adam`: number of epochs for Adam optimizer
- `hidden`: hidden layers and nodes (e.g., `[8]` for 1 hidden layer with 8 nodes)

These are the most relevant parameters for this notebook, but additional parameters may be set, which can be found in the documentation for `NNCompParams`.

Note that internally the data is split into training and validation portions, then shuffled. This is time series data, so if another model type, such as a recurrent neural network, is used, then the data should be kept in sequential order instead (no shuffling).

In [ ]:
model_type  = :m2b
η_adam      = 0.001
epoch_adam  = 100
hidden      = [8];


The neural network-based compensation parameters (type of `NNCompParams`) are provided to (and returned by) the training function. These take default values, unless they are specified.

In [ ]:
comp_params = NNCompParams(features_setup = features,
                           model_type     = model_type,
                           y_type         = y_type,
                           use_mag        = use_mag,
                           use_vec        = use_vec,
                           terms_A        = terms_A,
                           sub_diurnal    = sub_diurnal,
                           sub_igrf       = sub_igrf,
                           norm_type_x    = norm_type_x,
                           norm_type_y    = norm_type_y,
                           TL_coef        = TL_d_4,
                           η_adam         = η_adam,
                           epoch_adam     = epoch_adam,
                           hidden         = hidden);


The neural network model is then trained based on the specified parameters, which can of course be modified.

In [ ]:
(comp_params,y_train,y_train_hat,err_train,feats) =
    comp_train(comp_params,lines_train,df_all,df_flight,df_map);


We next test the performance on the held-out flight `line` using the `comp_test` convenience function. Note that there is also a `comp_train_test` convenience function that does both.

In [ ]:
(_,y_test_hat,_) =
    comp_test(comp_params,[line],df_all,df_flight,df_map);


### Model comparison

We are now in a position to compare the errors in the uncompensated, Tolles-Lawson compensated, and model 2b compensated signals. The model 2b results ameliorate the signal excursions that are present in the uncompensated and Tolles-Lawson compensated readings. 

Note that the `detrend` function helps remove any persistent bias in the signal, which does not affect the navigation error. Also note that since we selected `y_type = :d` in the NN compensation parameters (`NNCompParams`), we treat the output as the aircraft component that must be subtracted from the total scalar signal.

In [ ]:
tt = (xyz.traj.tt[ind] .- xyz.traj.tt[ind][1]) / 60;
di = (xyz.diurnal + xyz.igrf)[ind] # diurnal & core (IGRF)
p8 = plot(xlab="time [min]", ylab="magnetic field [nT]");
plot!(p8, tt, detrend(mag_1_sgl             - di, mean_only=true), lab="truth");
plot!(p8, tt, detrend(mag_4_uc              - di, mean_only=true), lab="uncompensated");
plot!(p8, tt, detrend(mag_4_c               - di, mean_only=true), lab="Tolles-Lawson");
plot!(p8, tt, detrend(mag_4_uc - y_test_hat - di, mean_only=true), lab="model 2b")


In [ ]:
println("raw σ:     ",round(Int,std( mag_4_uc               - mag_1_sgl))," nT")
println("TL σ:      ",round(Int,std( mag_4_c                - mag_1_sgl))," nT")
println("TL + NN σ: ",round(Int,std((mag_4_uc - y_test_hat) - mag_1_sgl))," nT")


## 6. Fine-tune the model

This model performs fairly well, but perhaps it could be improved by switching to model 2c (`:m2c`), which also tunes the Tolles-Lawson coefficients.

In [ ]:
comp_params = NNCompParams(comp_params,
                           model_type = :m2c,
                           epoch_adam = 50);
(comp_params,y_train,y_train_hat,err_train,y_test,y_test_hat,err_test,feats) =
    comp_train_test(comp_params,lines_train,[line],df_all,df_flight,df_map);


## 7. Present the solution

Document and present with clear visualizations and easy to remember statements
- What you have learned
- What worked and what did not
- What assumptions were made
- What system limitations exist

In [ ]:
p9 = plot(xlab="time [min]", ylab="magnetic field [nT]");
plot!(p9, tt, detrend(mag_1_sgl             - di, mean_only=true), lab="truth");
plot!(p9, tt, detrend(mag_4_uc              - di, mean_only=true), lab="uncompensated");
plot!(p9, tt, detrend(mag_4_c               - di, mean_only=true), lab="Tolles-Lawson");
plot!(p9, tt, detrend(mag_4_uc - y_test_hat - di, mean_only=true), lab="model 2c")


In [ ]:
println("raw σ:     ",round(Int,std( mag_4_uc               - mag_1_sgl))," nT")
println("TL σ:      ",round(Int,std( mag_4_c                - mag_1_sgl))," nT")
println("TL + NN σ: ",round(Int,std((mag_4_uc - y_test_hat) - mag_1_sgl))," nT")


### So what?

Here are the final model results.

Model             | Mag Error [nT]
----------------- | --------------
Uncompensated     | 405
Tolles-Lawson     | 134
TL + NN, model 2b | 67
TL + NN, model 2c | 63

With the error level achieved here, MagNav should be possible using an onboard magnetometer! We can pass the compensated values from this model into the navigation algorithm to assess its performance.

## 8. Launch, monitor, and maintain the system

Get the solution ready for launch
- Polish the code
- Write documentation and unit tests
- Load onto a device on the next data collection flight!

# Other Ideas

There are many other ideas that could be tried, such as:

- Explore the data in different ways such as autocorrelation analysis to gain additional insights.
- Experiment with other regression algorithms such as support vector machines, kernel ridge regression, or recurrent neural networks.
- Create a similar end-to-end machine learning project using a different magnetometer.
- Instead of using a hold-out validation dataset, use cross-validation, which trains models on different subsets of the dataset to estimate performance.

To help with this, the `x` data matrix and `y` target vector can be extracted directly, which is normally done internally to reduce memory use. Note that uncommenting below may cause the notebook to slow down significantly.

In [ ]:
# (A_train,x_train,y_train,_,_,l_segs_train) =
#     get_Axy(lines_train,df_all,df_flight,df_map,features;
#             y_type      = y_type,
#             use_mag     = use_mag,
#             use_vec     = use_vec,
#             terms_A     = terms_A,
#             sub_diurnal = sub_diurnal,
#             sub_igrf    = sub_igrf);


In [ ]:
# (A_test,x_test,y_test,_,_,l_segs_test) =
#     get_Axy([line],df_all,df_flight,df_map,features;
#             y_type      = y_type,
#             use_mag     = use_mag,
#             use_vec     = use_vec,
#             terms_A     = terms_A,
#             sub_diurnal = sub_diurnal,
#             sub_igrf    = sub_igrf);


These datasets can be normalized using the `norm_sets` function.

In [ ]:
# (A_bias,A_scale,A_train_norm,A_test_norm) = norm_sets(A_train,A_test;norm_type=:none);
# (x_bias,x_scale,x_train_norm,x_test_norm) = norm_sets(x_train,x_test;norm_type=norm_type_x);
# (y_bias,y_scale,y_train_norm,y_test_norm) = norm_sets(y_train,y_test;norm_type=norm_type_y);


Go at it!